In [7]:
import numpy as np
import pandas as pd
import simpy
import time

pd.options.mode.chained_assignment = None  # default='warn'

In [8]:
class Jobs():
    def do_jobs(self):
        print("Doing Jobs...")

## Fixed partition

In [9]:
# Generate random indices
NUM_JOBS = 30

In [10]:
def generate_fixed_partition_dataframe(num_jobs):
    random_memory_locations = np.random.randint(1, 10000, NUM_JOBS)
    random_job_status = np.random.randint(0, 2, NUM_JOBS)
    random_memory_block_size = np.random.randint(1, 1000, NUM_JOBS)
    return pd.DataFrame({"Memory Address": random_memory_locations, 
                            "Memory Block Size": random_memory_block_size,
                            "Job Status": random_job_status})

In [16]:
df = generate_fixed_partition_dataframe(NUM_JOBS)
df

,Memory Address,Memory Block Size,Job Status
0,7921,664,1
1,7886,890,1
2,9200,713,1
3,6586,410,0
4,9670,241,1
5,5080,462,0
6,4236,769,1
7,5322,870,1
8,381,877,0
9,799,926,1


In [21]:
def deallocate_fixed(df):
    deallocated_memory_count = 0
    for i in range(len(df)):
        if df["Job Status"].loc[i] == 1:
            # Simulating Fixed partition deallocation
            print(f"Deallocating Memory for Memory Address: {df['Memory Address'].loc[i]}...")
            time.sleep(2)
            df["Job Status"].loc[i] = 0
            deallocated_memory_count += 1
    print(f"Total memory deallocated: {deallocated_memory_count}")

deallocate_fixed(df)

Deallocating Memory for Memory Address: 7921...
Deallocating Memory for Memory Address: 7886...
Deallocating Memory for Memory Address: 9200...
Deallocating Memory for Memory Address: 9670...
Deallocating Memory for Memory Address: 4236...
Deallocating Memory for Memory Address: 5322...
Deallocating Memory for Memory Address: 799...
Deallocating Memory for Memory Address: 5961...
Deallocating Memory for Memory Address: 9450...
Deallocating Memory for Memory Address: 7603...
Deallocating Memory for Memory Address: 9679...
Deallocating Memory for Memory Address: 2409...
Deallocating Memory for Memory Address: 9378...
Deallocating Memory for Memory Address: 5997...
Deallocating Memory for Memory Address: 6131...
Deallocating Memory for Memory Address: 8729...
Deallocating Memory for Memory Address: 3549...
Deallocating Memory for Memory Address: 9421...
Total memory deallocated: 18


## Dynamic Partition

In [201]:
def generate_dynamic_partition_dataframe(num_jobs, case1=False, case2=False, case3=False):
    random_memory_locations = sorted(np.random.randint(1, 10000, num_jobs))
    random_job_status = np.random.randint(0, 2, num_jobs)
    random_memory_block_size = np.random.randint(1, 1000, num_jobs)
    
    if case1:
        random_job_status[-1] = 0
    
    return pd.DataFrame({"Memory Address": random_memory_locations, 
                         "Memory Block Size": random_memory_block_size,
                         "Job Status": random_job_status})

### Case 1

In [202]:
# Generate random dataframe
df_case_1 = generate_dynamic_partition_dataframe(30, case1=True)
df_case_1

,Memory Address,Memory Block Size,Job Status
0,440,473,1
1,758,124,1
2,882,147,1
3,1175,673,0
4,1346,500,0
5,1791,461,1
6,2009,764,0
7,2052,619,1
8,2440,164,1
9,2828,377,0


In [203]:
def deallocate_dynamic_case_1(df):
    deallocated_memory_count = 0
    iterations = 1
    
    while df["Job Status"].nunique() != 1:
        print(f"Iteration Number: {iterations}")
        rows_to_drop = []
        for i in range(len(df) - 1):
            if df["Job Status"].loc[i] == 1 and df["Job Status"].loc[i + 1] == 0:
                # Simulating case 1 dyanmic partition deallocation
                print(f"Deallocating Memory for Memory Address: {df['Memory Address'].loc[i]}...")
                print(f"Joining Memory Address {df['Memory Address'].loc[i]} and {df['Memory Address'].loc[i + 1]}")
                
                # Freeing job status
                df["Job Status"].loc[i] = 0

                # Joining memory address block sizes
                df["Memory Block Size"].loc[i] += df["Memory Block Size"].loc[i+1]

                # Putting the next memory address to rows to drop
                rows_to_drop.append(i + 1)

#                 time.sleep(2)
                deallocated_memory_count += 1
        print(f"Total memory deallocated: {deallocated_memory_count}")
        df.drop(rows_to_drop, inplace=True)
        df.reset_index(inplace=True, drop=True)
        iterations +=1

In [204]:
deallocate_dynamic_case_1(df_case_1)

Iteration Number: 1
Deallocating Memory for Memory Address: 882...
Joining Memory Address 882 and 1175
Deallocating Memory for Memory Address: 1791...
Joining Memory Address 1791 and 2009
Deallocating Memory for Memory Address: 2440...
Joining Memory Address 2440 and 2828
Deallocating Memory for Memory Address: 3598...
Joining Memory Address 3598 and 3764
Deallocating Memory for Memory Address: 4145...
Joining Memory Address 4145 and 5082
Deallocating Memory for Memory Address: 6773...
Joining Memory Address 6773 and 6911
Deallocating Memory for Memory Address: 8066...
Joining Memory Address 8066 and 8353
Deallocating Memory for Memory Address: 8581...
Joining Memory Address 8581 and 9284
Total memory deallocated: 8
Iteration Number: 2
Deallocating Memory for Memory Address: 758...
Joining Memory Address 758 and 882
Deallocating Memory for Memory Address: 2052...
Joining Memory Address 2052 and 2440
Deallocating Memory for Memory Address: 8533...
Joining Memory Address 8533 and 8581
To

In [205]:
df_case_1

,Memory Address,Memory Block Size,Job Status
0,440,1417,0
1,1346,500,0
2,1791,1225,0
3,2052,1160,0
4,3546,318,0
5,3598,364,0
6,4145,175,0
7,5974,923,0
8,6117,859,0
9,6137,736,0


### Case 2: Joining Three Free Blocks

In [258]:
df_case_2 = generate_dynamic_partition_dataframe(NUM_JOBS, case2=True)
df_case_2

,Memory Address,Memory Block Size,Job Status
0,140,734,0
1,637,763,1
2,969,380,1
3,1123,512,0
4,2074,732,1
5,2691,865,0
6,3383,471,0
7,3524,697,1
8,3566,565,1
9,3701,245,1


In [259]:
def deallocate_dynamic_case_2(df):
    deallocated_memory_count = 0
    iterations = 1
    
 
    rows_to_drop = []
    for i in range(1, len(df) - 2):
        if df["Job Status"].loc[i] == 1 and df["Job Status"].loc[i + 1] == 0 and df["Job Status"].loc[i - 1] == 0:
            # Simulating case 2 dyanmic partition deallocation
            print(f"Deallocating Memory for Memory Address: {df['Memory Address'].loc[i]}...")
            print(f"Joining Memory Address {df['Memory Address'].loc[i-1]}, {df['Memory Address'].loc[i]} and {df['Memory Address'].loc[i+1]}")
                
            # Freeing job status
            df["Job Status"].loc[i] = 0

            # Joining memory address block sizes
            df["Memory Block Size"].loc[i-1] += df["Memory Block Size"].loc[i]
            df["Memory Block Size"].loc[i-1] += df["Memory Block Size"].loc[i+1]

            # Putting the next memory address to rows to drop
            rows_to_drop.append(i + 1)
                
            # Replace current memory location to a null entry
            df.loc[i, "Memory Address"] = "*"
            df.loc[i, "Memory Block Size"] = 0
            df.loc[i, "Job Status"] = None
            df.loc[i+1, "Job Status"] = 1

#               time.sleep(2)
            deallocated_memory_count += 1
    print(f"Total memory deallocated: {deallocated_memory_count}")
    df.drop(rows_to_drop, inplace=True)
    df.reset_index(inplace=True, drop=True)
    iterations +=1

In [260]:
deallocate_dynamic_case_2(df_case_2)

Deallocating Memory for Memory Address: 2074...
Joining Memory Address 1123, 2074 and 2691
Deallocating Memory for Memory Address: 6292...
Joining Memory Address 6146, 6292 and 6304
Total memory deallocated: 2


In [261]:
df_case_2

,Memory Address,Memory Block Size,Job Status
0,140,734,0.0
1,637,763,1.0
2,969,380,1.0
3,1123,2109,0.0
4,*,0,NaN
5,3383,471,0.0
6,3524,697,1.0
7,3566,565,1.0
8,3701,245,1.0
9,4667,316,0.0
